# Parcial 1 - Parte práctica

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
predTarjeta = pd.read_csv('Prediccion_Tarjeta.csv')
datosTarjeta = pd.read_csv('Datos_Tarjeta_Credito.csv')

### Procesamiento de datos

In [3]:
predTarjeta.shape

(3000, 23)

In [7]:
datosTarjeta.shape

(27000, 24)

In [8]:
predTarjeta.head(5)

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
0,60000,1,2,1,49,1,-2,-2,-2,-2,...,0,0,0,0,0,0,0,0,0,0
1,60000,1,3,2,44,0,0,0,0,0,...,59198,31437,32113,33591,2300,2600,1600,1200,2000,1300
2,50000,2,3,1,48,0,0,0,0,-1,...,7857,3181,2430,780,1800,1506,6,2430,780,0
3,50000,1,3,1,48,0,0,0,0,0,...,42248,33855,28064,29660,5000,2000,2500,2500,2000,2500
4,130000,2,2,2,27,0,0,0,0,0,...,110317,108099,106028,104253,5578,4000,3863,3831,4136,5385


In [9]:
datosTarjeta.head(5)

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,y
0,490000,2,1,2,28,0,0,0,0,0,...,132268,118899,79223,10041,7415,7025,44324,3119,2780,0
1,140000,1,1,2,25,-1,-1,-1,-1,-1,...,643,2561,1440,1844,1311,643,2561,1440,1582,0
2,100000,1,1,2,28,0,0,0,0,0,...,12233,6741,-24,1400,1506,1209,0,0,0,0
3,500000,1,1,1,39,0,0,0,0,0,...,27843,28926,11931,30006,23016,566,22323,11931,10683,0
4,550000,2,1,1,53,0,0,0,0,0,...,214030,218096,210264,7110,7353,7534,7385,7253,15000,0


### Modelado